### Import Libraries

In [1]:
!pip install -q tensorflow

In [2]:
import pandas as pd
import numpy as np
import os
import re
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from numpy.linalg import norm

In [4]:
from scipy import stats
from scipy.spatial import distance

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
import tensorflow
import tensorflow.keras

In [7]:
from sentence_transformers import util

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# put path of your project folder here

proj_path = '/content/drive/MyDrive/anime_recommendation_system'
os.chdir(proj_path)

### Load Data

In [10]:
user = pd.read_pickle('datasets/created_datasets/user.pkl')
anime = pd.read_pickle('datasets/created_datasets/anime.pkl')

In [11]:
content_df_path = 'datasets/created_datasets/content_df_model.pkl'

with open(content_df_path, 'rb') as f:
    content_df = pickle.load(f)

In [12]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59516292 entries, 0 to 59516291
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    uint32
 1   anime_id   uint16
 2   rating     uint8 
 3   user_enc   Int32 
 4   anime_enc  uint16
dtypes: Int32(1), uint16(2), uint32(1), uint8(1)
memory usage: 794.6 MB


In [13]:
user.head(7)

,user_id,anime_id,rating,user_enc,anime_enc
0,0,67,9,0,0
1,0,6702,7,0,1
2,0,242,10,0,2
3,0,24,9,0,3
4,0,4722,8,0,4
5,0,6098,6,0,5
6,0,3125,9,0,6


In [14]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6356 entries, 0 to 6355
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         6356 non-null   int64 
 1   Name           6356 non-null   object
 2   Score          6356 non-null   object
 3   Genres         6356 non-null   object
 4   English name   6356 non-null   object
 5   Japanese name  6356 non-null   object
 6   Type           6356 non-null   object
 7   Episodes       6356 non-null   object
 8   Aired          6356 non-null   object
 9   Premiered      6356 non-null   object
 10  Producers      6356 non-null   object
 11  Licensors      6356 non-null   object
 12  Studios        6356 non-null   object
 13  Source         6356 non-null   object
 14  Duration       6356 non-null   object
 15  Rating         6356 non-null   object
 16  Ranked         6356 non-null   object
 17  Popularity     6356 non-null   int64 
 18  Members        6356 non-null

In [15]:
content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MAL_ID           5144 non-null   int64 
 1   final_plot       5144 non-null   object
 2   Name             5144 non-null   object
 3   plot_embeddings  5144 non-null   object
dtypes: int64(1), object(3)
memory usage: 160.9+ KB


In [16]:
content_df.head()

,MAL_ID,final_plot,Name,plot_embeddings
0,1,"In the year 2071, roughly fifty years after an...",Cowboy Bebop,"[-0.65085495, 0.08365378, -0.5011058, -0.04219..."
1,5,"A few days before Halloween on Mars, Faye witn...",Cowboy Bebop: Tengoku no Tobira,"[-0.7295263, -0.24117824, 0.5099685, -0.530645..."
2,6,"In a distant future, a man known as ""Vash the ...",Trigun,"[-0.9758292, -0.2694932, 0.54436916, -0.191350..."
3,7,"Robin Sena is a ""craft user"", born in Japan an...",Witch Hunter Robin,"[-1.0255445, -0.21402474, 0.20005345, 0.112862..."
4,8,"Set in the Dark Ages, a term referring to the ...",Bouken Ou Beet,"[-0.4451655, -0.50909936, -0.063745886, 0.1937..."


In [17]:
content_df = content_df[content_df['MAL_ID'].isin(anime['MAL_ID'].unique())]
content_df.reset_index(inplace = True, drop = True)
content_df.head()

,MAL_ID,final_plot,Name,plot_embeddings
0,1,"In the year 2071, roughly fifty years after an...",Cowboy Bebop,"[-0.65085495, 0.08365378, -0.5011058, -0.04219..."
1,5,"A few days before Halloween on Mars, Faye witn...",Cowboy Bebop: Tengoku no Tobira,"[-0.7295263, -0.24117824, 0.5099685, -0.530645..."
2,6,"In a distant future, a man known as ""Vash the ...",Trigun,"[-0.9758292, -0.2694932, 0.54436916, -0.191350..."
3,7,"Robin Sena is a ""craft user"", born in Japan an...",Witch Hunter Robin,"[-1.0255445, -0.21402474, 0.20005345, 0.112862..."
4,8,"Set in the Dark Ages, a term referring to the ...",Bouken Ou Beet,"[-0.4451655, -0.50909936, -0.063745886, 0.1937..."


### Preprocessing Data

Before we build our features for Recommendation, we need some analysis of some <br> columns, also creating new columns and dataframes which will be helpful

#### 1. Cleaning 'Genres' Column

In [18]:
# Let's inspect Genres column in anime, as it is an important column
anime['Genres'].head()

,Genres
0,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"
1,"Action, Drama, Mystery, Sci-Fi, Space"
2,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen"
3,"Action, Mystery, Police, Supernatural, Drama, ..."
4,"Adventure, Fantasy, Shounen, Supernatural"


In [19]:
# we split the genres where ',' is present,
# and then split them again based on ' '
# so that we are sure comma is present in every data properly
# And if comma is not present like for Ex. Shounen Action, we ccovert it to
# Shounen, Action or better we will use '|' as separator

genre_insp = pd.Series((' ').join(anime['Genres']).split(', ')).str.split(' ')
genre_insp[genre_insp.apply(lambda x: True if len(x) > 1 else False)].value_counts().head(25)

,count
"[Shounen, Action]",413
"[Super, Power]",234
"[Shounen, Comedy]",214
"[Martial, Arts]",173
"[Seinen, Action]",172
"[Slice, of, Life]",158
"[Fantasy, Action]",150
"[Supernatural, Action]",150
"[Shounen, Adventure]",142
"[Sci-Fi, Action]",142


In [20]:
anime['Genres_edited'] = anime['Genres'].copy()

anime['Genres_edited'] = anime['Genres_edited'].str.replace('Slice of Life', 'Slice-of-Life')
anime['Genres_edited'] = anime['Genres_edited'].str.replace('Martial Arts', 'Martial-Arts')
anime['Genres_edited'] = anime['Genres_edited'].str.replace('Super Power', 'Super-Power')

In [21]:
def space_split_genres(genres):
    '''
    Function to assure no space between words in genres and using '|' as separator
    Input: str
    Output: str
    '''

    # 'Comedy, Shounen Aciton will convert to 'Action|Comedy|Shounen'
    genres_edited = genres.copy()
    for i in range(len(genres)):
        if ' ' in genres[i]:
            genres_edited = genres_edited[0:i] + genres[i].split(' ') + genres[i+1:]
    return ('|').join(sorted(genres_edited))

In [22]:
anime['Genres_edited'] = anime['Genres_edited'].str.split(', ').apply(space_split_genres)

In [23]:
anime['Genres_edited'].head(5)

,Genres_edited
0,Action|Adventure|Comedy|Drama|Sci-Fi|Space
1,Action|Drama|Mystery|Sci-Fi|Space
2,Action|Adventure|Comedy|Drama|Sci-Fi|Shounen
3,Action|Drama|Magic|Mystery|Police|Supernatural
4,Adventure|Fantasy|Shounen|Supernatural


In [24]:
genre_value_counts = pd.Series(('|').join(anime['Genres_edited']).split('|')).value_counts()
genre_value_counts

,count
Comedy,3036
Action,2241
Fantasy,1508
Drama,1476
Shounen,1419
Adventure,1389
Sci-Fi,1362
Romance,1356
School,1106
Supernatural,961


In [25]:
# Let's grab names of the genres present, we are ignoring 'Unknown' genre

genres_list = genre_value_counts.index.tolist()[:-1]
print(genres_list)

['Comedy', 'Action', 'Fantasy', 'Drama', 'Shounen', 'Adventure', 'Sci-Fi', 'Romance', 'School', 'Supernatural', 'Slice-of-Life', 'Magic', 'Ecchi', 'Seinen', 'Mecha', 'Mystery', 'Shoujo', 'Super-Power', 'Historical', 'Military', 'Sports', 'Harem', 'Horror', 'Psychological', 'Parody', 'Music', 'Space', 'Demons', 'Kids', 'Martial-Arts', 'Game', 'Police', 'Ai', 'Vampire', 'Samurai', 'Thriller', 'Josei', 'Dementia', 'Cars', 'Yaoi']


####2. Building Encoded dataframe of genres

In [26]:
# Let's create Encoded dataframe of genres

genres_mlb = MultiLabelBinarizer(classes = genres_list)
anime_genres_mlb = anime[['MAL_ID', 'Genres_edited']].copy()

anime_genres_mlb_temp = pd.DataFrame(genres_mlb.fit_transform(anime_genres_mlb['Genres_edited'].str.split('|')), columns = genres_mlb.classes_)
anime_genres_mlb = anime_genres_mlb.join(anime_genres_mlb_temp)
anime_genres_mlb.head()

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Unknown'] will be ignored
  warnings.warn(


,MAL_ID,Genres_edited,Comedy,Action,Fantasy,Drama,Shounen,Adventure,Sci-Fi,Romance,...,Game,Police,Ai,Vampire,Samurai,Thriller,Josei,Dementia,Cars,Yaoi
0,1,Action|Adventure|Comedy|Drama|Sci-Fi|Space,1,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5,Action|Drama|Mystery|Sci-Fi|Space,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,6,Action|Adventure|Comedy|Drama|Sci-Fi|Shounen,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,7,Action|Drama|Magic|Mystery|Police|Supernatural,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,8,Adventure|Fantasy|Shounen|Supernatural,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


####3. Building aggregrated dataframes for user and anime data

In [27]:
# Let's create user_agg (i.e. user aggregrate)

user_agg = user.groupby('user_id').agg(animes_watched = ('anime_id', 'count'),
                                      user_avg_rating = ('rating', 'mean')).reset_index()
user_agg.head()

,user_id,animes_watched,user_avg_rating
0,0,40,7.725000
1,1,117,8.051282
2,2,46,8.521739
3,3,317,7.611987
4,4,120,7.633333


In [28]:
# Create anime aggregrate dataframe

anime_agg = user.groupby('anime_id')['rating'].agg(['mean', 'count']).reset_index()
anime_agg.columns = ['anime_id', 'anime_avg_rating', 'total_ratings_anime']
anime_agg.head()

,anime_id,anime_avg_rating,total_ratings_anime
0,1,8.668267,86648
1,5,8.318500,33617
2,6,8.167207,50279
3,7,7.265829,11624
4,8,6.935718,1789


####4. Building parameter 'rating_diff'

<b>rating_diff</b> will reflect how much the user enjoyed the anime \
keeping in mind both the genreal consensus regarding the anime
and average rating given by user

In [29]:
user['anime_avg_rating'] = user['anime_id'].map(anime_agg.set_index('anime_id')['anime_avg_rating']).astype('float32')
user['user_avg_rating'] = user['user_id'].map(user_agg.set_index('user_id')['user_avg_rating']).astype('float32')

In [30]:
user.head()

,user_id,anime_id,rating,user_enc,anime_enc,anime_avg_rating,user_avg_rating
0,0,67,9,0,0,7.557934,7.725
1,0,6702,7,0,1,7.646535,7.725
2,0,242,10,0,2,7.378310,7.725
3,0,24,9,0,3,7.976726,7.725
4,0,4722,8,0,4,8.248917,7.725


In [31]:
# Creating rating_diff, which will be average of rating_diff_wrt_anime and rating_diff_wrt_user
# rating_diff will reflect how much the user enjoyed the anime
# (keeping in mind both the genreal consensus regarding the anime and average rating given by user)

user['rating_diff_wrt_anime'] = user['rating'] - user['anime_avg_rating']
user['rating_diff_wrt_user'] = user['rating'] - user['user_avg_rating']
user['rating_diff'] = 0.5*(user['rating_diff_wrt_anime'] + user['rating_diff_wrt_user']).astype('float32')

In [32]:
user.drop(['anime_avg_rating', 'user_avg_rating', 'rating_diff_wrt_anime', 'rating_diff_wrt_user'], inplace = True, axis = 1)

In [33]:
user.head()

,user_id,anime_id,rating,user_enc,anime_enc,rating_diff
0,0,67,9,0,0,1.358533
1,0,6702,7,0,1,-0.685767
2,0,242,10,0,2,2.448345
3,0,24,9,0,3,1.149137
4,0,4722,8,0,4,0.013042


####5. Extracting Origin Year of the anime from the 'Aired' Column

In [34]:
anime['Aired'].head(10)

,Aired
0,"Apr 3, 1998 to Apr 24, 1999"
1,"Sep 1, 2001"
2,"Apr 1, 1998 to Sep 30, 1998"
3,"Jul 2, 2002 to Dec 24, 2002"
4,"Sep 30, 2004 to Sep 29, 2005"
5,"Apr 6, 2005 to Mar 19, 2008"
6,"Apr 15, 2005 to Sep 27, 2005"
7,"Sep 11, 2002 to Sep 10, 2003"
8,"Apr 17, 2004 to Feb 18, 2006"
9,"Apr 7, 2004 to Sep 28, 2005"


In [35]:
def year_finding(aired):

    '''
    extracting year from Aired column
    Input: str
    Output: int
    '''
    if aired == 'Unknown':
        return None
    match = re.search(r'([1-2][0-9]{3})', aired)
    if match is not None:
        return int(match.group(0))
    else:
        return -1

In [36]:
anime['Origin_year'] = anime['Aired'].apply(lambda x: year_finding(str(x))).astype('Int32')

In [37]:
anime['Origin_year'].value_counts().sort_index().head(10)

,count
Origin_year,
1917,1
1918,1
1929,1
1930,1
1931,3
1943,1
1958,1
1962,3
1963,2


In [38]:
anime['Origin_year'].isna().sum()

np.int64(1)

In [39]:
anime[anime['Origin_year'].isna()]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_edited,Origin_year
5060,33187,Katsudou Shashin,5.48,Unknown,Unknown,活動写真,Movie,1,Unknown,Unknown,...,84.0,245.0,407.0,956.0,202.0,165.0,155.0,261.0,Unknown,<NA>


In [40]:
anime.loc[anime['MAL_ID'] == 33187, 'Origin_year'] = 1907

In [41]:
anime[anime['Origin_year'].isna()]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_edited,Origin_year


In [43]:
anime['total_ratings_anime'] = anime['MAL_ID'].map(anime_agg.set_index('anime_id')['total_ratings_anime'])
sorted_indices = anime.sort_values(by=['Popularity', 'total_ratings_anime'], ascending=[True, False]).index
anime.loc[sorted_indices, 'Popularity_adjusted'] = range(1, len(anime) + 1)
anime['Popularity_adjusted'] = anime['Popularity_adjusted'].astype('int')
anime.drop(columns=['total_ratings_anime'], inplace=True)

In [44]:
anime['Popularity_adjusted'].head()

,Popularity_adjusted
0,39
1,517
2,202
3,1459
4,4174


## Building Recommendation Features

In [45]:
def get_anime_id_from_name(name):
    try:
        return anime[anime['Name'] == name]['MAL_ID'].values[0]
    except:
        try:
            return anime[anime['English name'] == name]['MAL_ID'].values[0]
        except:
            print('Anime not found')
            return None

In [46]:
def get_anime_name_from_id(anime_id):
    try:
        return anime[anime['MAL_ID'] == anime_id]['Name'].values[0]
    except:
        print('Anime not found')
        return None

In [47]:
user_ids_model = user[user['user_enc'].notna()]['user_id'].unique().tolist()
user_ids_complete = user['user_id'].unique().tolist()

In [48]:
def get_random_user_id(user_ids = user_ids_model):
    return np.random.choice(user_ids)

#### Function for filtering Recommendations

In [49]:
def filter_recommendations(recommendations, anime, anime_agg, Type_preferred = None, Genres_preferred = None, Origin_year_range = None,
                           Popularity_range = None, min_anime_rating = None):

      '''
      Function to filter recommendations based on user preferences
      Input:
      recommendations: pd.DataFrame
      anime: pd.DataFrame
      anime_agg: pd.DataFrame
      Type_preferred: list          Ex. ['TV', 'Movie']
      Genres_preferred: list        Ex. ['Action', 'Adventure']
      Origin_year_range: list       Ex. [1990, 2000]
      Popularity_range: list        Ex. [100, 200]
      min_anime_rating: float       Ex. 8.5
      Output: pd.DataFrame
      '''
      if recommendations is None:
          print('No recommendations found')
          return None

      if 'anime_id' in recommendations.columns:
          recommendations = recommendations.merge(anime[['MAL_ID', 'Name', 'English name', 'Type', 'Genres_edited', 'Origin_year',
                                                    'Popularity_adjusted']], left_on = 'anime_id', right_on = 'MAL_ID', how = 'left')
          recommendations = recommendations.drop('anime_id', axis = 1)
      else:
          recommendations = recommendations.merge(anime[['MAL_ID', 'Name', 'English name', 'Type', 'Genres_edited', 'Origin_year',
                                                    'Popularity_adjusted']], how = 'left')

      if 'anime_avg_rating' not in recommendations.columns:
          recommendations = recommendations.merge(anime_agg[['anime_id', 'anime_avg_rating']], left_on = 'MAL_ID', right_on = 'anime_id', how = 'left')
          recommendations = recommendations.drop('anime_id', axis = 1)

      if Type_preferred is not None:
          recommendations = recommendations[recommendations['Type'].isin(Type_preferred)]
      if Genres_preferred is not None:
          recommendations = recommendations[recommendations['Genres_edited'].str.split('|').apply(lambda x: True if set(Genres_preferred).issubset(x) else False)]
      if Origin_year_range is not None:
          recommendations = recommendations[(recommendations['Origin_year'] >= Origin_year_range[0]) & (recommendations['Origin_year'] <= Origin_year_range[1])]
      if Popularity_range is not None:
          recommendations = recommendations[(recommendations['Popularity_adjusted'] >= Popularity_range[0]) & (recommendations['Popularity_adjusted'] <= Popularity_range[1])]
      if min_anime_rating is not None:
          recommendations = recommendations[recommendations['anime_avg_rating'] >= min_anime_rating]

      recommendations = recommendations.reset_index(drop = True)

      recommendations = recommendations[['MAL_ID', 'Name', 'English name', 'Genres_edited', 'Type',  'Origin_year',
                                         'anime_avg_rating', 'Popularity_adjusted']]
      return recommendations


### 1.Model Based Features

In [50]:
# Let's load our trained model

checkpoint_filepath = 'model/checkpoint.model.keras'

model = tensorflow.keras.models.load_model(checkpoint_filepath)

In [51]:
# Let's load our encodings of the categorical data

user2user_encoded = pickle.load(open('datasets/created_datasets/encoded_dictionary/user2user_encoded.pkl', 'rb'))
user_encoded2user = pickle.load(open('datasets/created_datasets/encoded_dictionary/user_encoded2user.pkl', 'rb'))
anime2anime_encoded = pickle.load(open('datasets/created_datasets/encoded_dictionary/anime2anime_encoded.pkl', 'rb'))
anime_encoded2anime = pickle.load(open('datasets/created_datasets/encoded_dictionary/anime_encoded2anime.pkl', 'rb'))

In [52]:
encoded_dictionary = {'user2user_encoded': user2user_encoded,
                      'user_encoded2user': user_encoded2user,
                      'anime2anime_encoded': anime2anime_encoded,
                      'anime_encoded2anime': anime_encoded2anime}

In [53]:
# Load weights from the model

def extract_weights(layer_name, model):
    weight_layer = model.get_layer(layer_name)
    weights = weight_layer.get_weights()[0]
    return weights

anime_weights = extract_weights('anime_embedding', model)
user_weights = extract_weights('user_embedding', model)

In [55]:
selected_user_id_1 = get_random_user_id()
print(selected_user_id_1)

322554


#### 1.1 Recommendation based on Rating Prediction by Model

In [56]:
def model_rec_based_on_rating_prediction(selected_user_id, anime, user, encoded_dictionary, model):
    """
    Generates anime recommendations for a user based on predicted ratings from a deep learning model.

    Parameters:
    -----------
    selected_user_id : int
        The ID of the user for whom recommendations are to be generated.
    anime : DataFrame
    user : DataFrame
    encoded_dictionary : dict
        A dictionary containing:
            - 'anime2anime_encoded': mapping from actual anime IDs to encoded indices.
            - 'anime_encoded2anime': reverse mapping from encoded indices back to actual anime IDs.
            - 'user2user_encoded': mapping from actual user IDs to encoded indices.
            - 'user_encoded2user': reverse mapping from encoded indices back to actual user IDs.
    model : keras.Model
        The trained collaborative filtering model that predicts ratings using user and anime embeddings.

    Returns:
    --------
    DataFrame
        A sorted dataframe of animes not yet watched by the user, along with their predicted ratings.
    """

    try:
        # Get encoded user ID from dictionary
        selected_user_id_enc = encoded_dictionary['user2user_encoded'].get(selected_user_id)
    except:
        print('User not found')
        return

    # Create a DataFrame of unwatched animes
    selected_user_id_animes_not_watched = pd.DataFrame(list(set(anime['MAL_ID'].unique()) - set(user[user['user_id'] == selected_user_id]['anime_id'].tolist())))
    selected_user_id_animes_not_watched.columns = ['anime_id']

    # Encode anime IDs using the provided dictionary
    selected_user_id_animes_not_watched['anime_enc'] = selected_user_id_animes_not_watched['anime_id'].map(encoded_dictionary['anime2anime_encoded'])

    # Predict ratings for all unwatched animes using the model
    predictions = model.predict([
        np.array([selected_user_id_enc] * len(selected_user_id_animes_not_watched)),  # user input
        np.array(selected_user_id_animes_not_watched['anime_enc'].tolist())            # anime input
    ])

    # Store predictions in the dataframe
    selected_user_id_animes_not_watched['rating_prediction'] = predictions

    # Sort animes based on predicted rating (highest first)
    recommendations = selected_user_id_animes_not_watched.sort_values(by='rating_prediction', ascending=False)

    return recommendations


In [57]:
recommendations = model_rec_based_on_rating_prediction(selected_user_id_1, anime, user, encoded_dictionary, model)
recommendations

195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


,anime_id,anime_enc,rating_prediction
4828,9253,130,9.504438
2845,3784,647,9.474910
3150,4565,1680,9.474340
30,44,1643,9.471206
2958,36862,1576,9.466930
...,...,...,...
5605,13405,3286,5.222404
4845,9342,4743,5.151887
2626,3287,2735,4.982163
384,413,661,4.975003


In [58]:
filter_recommendations(recommendations, anime, anime_agg, Type_preferred = ['TV', 'Movie'], Genres_preferred = ['Action', 'Adventure']).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist:Brotherhood,Action|Adventure|Comedy|Drama|Fantasy|Magic|Mi...,TV,2009,9.148895,3
1,12115,Berserk: Ougon Jidai-hen III - Kourin,Berserk:The Golden Age Arc III - The Advent,Action|Adventure|Demons|Drama|Fantasy|Horror|M...,Movie,2013,8.167687,939
2,2001,Tengen Toppa Gurren Lagann,Gurren Lagann,Action|Adventure|Comedy|Mecha|Sci-Fi,TV,2007,8.647373,46
3,33,Kenpuu Denki Berserk,Berserk,Action|Adventure|Demons|Drama|Fantasy|Horror|M...,TV,1997,8.354521,302
4,37521,Vinland Saga,Unknown,Action|Adventure|Drama|Historical|Seinen,TV,2019,8.630568,145
5,486,Kino no Tabi: The Beautiful World,Kino's Journey,Action|Adventure|Psychological|Slice-of-Life,TV,2003,8.326377,649
6,37991,JoJo no Kimyou na Bouken Part 5: Ougon no Kaze,JoJo's Bizarre Adventure:Golden Wind,Action|Adventure|Shounen,TV,2018,8.484070,211
7,2418,Stranger: Mukou Hadan,Sword of the Stranger,Action|Adventure|Historical|Samurai,Movie,2007,8.312847,560
8,23293,Lupin the IIIrd: Jigen Daisuke no Bohyou,Lupin III:Jigen's Gravestone,Action|Adventure|Drama|Seinen,Movie,2014,7.836842,3529
9,1924,Urusei Yatsura Movie 5: Kanketsu-hen,Unknown,Action|Adventure|Comedy|Drama|Romance|Sci-Fi,Movie,1988,7.521357,5617


#### 1.2 Recommendation using User-User Similarity via Embeddings

In [59]:
def model_rec_based_on_user_similarity(user_id, user_weights, encoded_dictionary, user, anime,
                                       sort_by='rating_mean', top_n_similar_users=100, user_threshold_for_rec=20):
    """
    Recommends anime to a user based on preferences of similar users using user embedding similarity.

    Parameters:
    -----------
    user_id : int
        The target user ID for whom we want to generate recommendations.
    user_weights : np.ndarray
        The embedding matrix of users from the trained deep learning model.
        Each row represents an encoded user.
    encoded_dictionary : dict
        Dictionary containing:
            - 'user2user_encoded': mapping from real user IDs to encoded indices.
            - 'user_encoded2user': reverse mapping from encoded indices to real user IDs.
            - 'anime2anime_encoded': mapping from real anime IDs to encoded indices.
            - 'anime_encoded2anime': reverse mapping from encoded indices to real anime IDs.
    user : pd.DataFrame
    anime : pd.DataFrame
    sort_by : str (default='rating_mean')
        The column to sort recommendations by. Options include 'rating_mean', 'users_watched' and 'rating_diff_mean'.
    top_n_similar_users : int (default=100)
        Number of most similar users to consider for making recommendations.
    user_threshold_for_rec : int (default=20)
        Minimum number of users who must have rated an anime for it to be considered as a recommendation.

    Returns:
    --------
    pd.DataFrame
        A DataFrame of recommended anime (not watched by the user) along with aggregated rating stats
        from similar users, sorted by the chosen metric.
    """

    try:
        # Get encoded index for the input user
        encoded_index = encoded_dictionary['user2user_encoded'].get(user_id)

        # normalize the embeddings, so cosine similarity with other
        # anime embeddings will simply be dot product
        user_weights_norm = user_weights/np.linalg.norm(user_weights, axis = 1).reshape((-1, 1))
        dists = np.dot(user_weights_norm, user_weights_norm[encoded_index])

        # Get indices of top N similar users (excluding the user themselves)
        sorted_dists = np.argsort(dists)
        top_n_similar_user_encoded_index = list(sorted_dists[-1:-(top_n_similar_users + 2):-1])
        top_n_similar_user_encoded_index.remove(encoded_index)

        # Decode back to original user IDs
        top_n_similar_user_ids = [encoded_dictionary['user_encoded2user'].get(i) for i in top_n_similar_user_encoded_index]

        # Filter data for these similar users
        relevant_user_data = user[user['user_id'].isin(pd.Series(top_n_similar_user_ids))]

        # Exclude animes already watched by the target user
        relevant_user_data = relevant_user_data[
            ~relevant_user_data['anime_id'].isin(user[user['user_id'] == user_id]['anime_id'])
        ]

        # Group by anime and calculate stats from ratings
        relevant_user_data_grouped = relevant_user_data.groupby('anime_id').agg(
            users_watched=('rating', 'count'),
            rating_mean=('rating', lambda x: stats.trim_mean(x, 0.05)),  # trim mean to reduce outlier impact
            rating_diff_mean=('rating_diff', lambda y: stats.trim_mean(y, 0.05))
        )

        # Clean up result
        relevant_user_data_grouped = relevant_user_data_grouped.reset_index()
        relevant_user_data_grouped['rating_diff_mean'] = relevant_user_data_grouped['rating_diff_mean'].astype(np.float64)

        # Filter animes rated by at least `user_threshold_for_rec` users
        relevant_user_data_grouped = relevant_user_data_grouped[
            relevant_user_data_grouped['users_watched'] >= user_threshold_for_rec
        ]

        # Sort by chosen metric
        relevant_user_data_grouped = relevant_user_data_grouped.sort_values(
            by=sort_by, ascending=False
        )

        return relevant_user_data_grouped

    except Exception as e:
        print(e)
        return


In [60]:
selected_user_id_2 = get_random_user_id()
selected_user_id_2

np.int64(229979)

In [61]:
rec_based_on_user_sim_via_model_embeddings = model_rec_based_on_user_similarity(
                                              selected_user_id_2, user_weights, encoded_dictionary, user, anime,
                                              sort_by = 'rating_diff_mean',
                                              top_n_similar_users=200, user_threshold_for_rec = 20)
rec_based_on_user_sim_via_model_embeddings.head(10)

,anime_id,users_watched,rating_mean,rating_diff_mean
4446,40028,26,9.750000,1.634553
450,820,20,9.333333,1.508893
2235,11061,133,9.479339,1.470650
5,21,92,9.095238,1.399301
2313,12115,22,8.800000,1.307529
4232,37991,37,8.942857,1.270496
15,33,44,8.825000,1.261117
3720,34096,26,9.083333,1.232222
3086,26055,56,8.769231,1.196819
2063,9969,42,9.131579,1.166868


In [62]:
filter_recommendations(rec_based_on_user_sim_via_model_embeddings, anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,40028,Shingeki no Kyojin: The Final Season,Attack on Titan Final Season,Action|Drama|Fantasy|Military|Mystery|Shounen|...,TV,2020,9.042170,120
1,820,Ginga Eiyuu Densetsu,Legend of the Galactic Heroes,Drama|Military|Sci-Fi|Space,OVA,1988,8.945480,632
2,11061,Hunter x Hunter (2011),Hunter x Hunter,Action|Adventure|Fantasy|Shounen|Super-Power,TV,2011,9.049059,12
3,21,One Piece,One Piece,Action|Adventure|Comedy|Drama|Fantasy|Shounen|...,TV,1999,8.487537,31
4,12115,Berserk: Ougon Jidai-hen III - Kourin,Berserk:The Golden Age Arc III - The Advent,Action|Adventure|Demons|Drama|Fantasy|Horror|M...,Movie,2013,8.167687,939
5,37991,JoJo no Kimyou na Bouken Part 5: Ougon no Kaze,JoJo's Bizarre Adventure:Golden Wind,Action|Adventure|Shounen,TV,2018,8.484070,211
6,33,Kenpuu Denki Berserk,Berserk,Action|Adventure|Demons|Drama|Fantasy|Horror|M...,TV,1997,8.354521,302
7,34096,Gintama.,Gintama Season 5,Action|Comedy|Historical|Parody|Samurai|Sci-Fi...,TV,2017,8.915591,728
8,26055,JoJo no Kimyou na Bouken Part 3: Stardust Crus...,JoJo's Bizarre Adventure:Stardust Crusaders - ...,Action|Adventure|Drama|Shounen|Supernatural,TV,2015,8.408956,159
9,9969,Gintama',Gintama Season 2,Action|Comedy|Historical|Parody|Samurai|Sci-Fi...,TV,2011,9.090924,362


#### 1.3 Recommendation using Anime-Anime Similarity via Embeddings

In [63]:
def model_rec_based_on_anime_similarity(name, anime_weights, encoded_dictionary):
    """
    Recommends similar animes based on learned anime embeddings from a collaborative filtering model.

    Parameters:
    -----------
    name : str
        The name of the input anime for which similar animes are to be recommended.
    anime_weights : np.ndarray
        The weight matrix (embeddings) of animes from the trained deep learning model.
        Each row corresponds to an encoded anime index.
    encoded_dictionary : dict
        A dictionary containing:
            - 'anime2anime_encoded': mapping from actual anime IDs to encoded indices.
            - 'anime_encoded2anime': reverse mapping from encoded indices back to actual anime IDs.
            - 'user2user_encoded': mapping from actual user IDs to encoded indices.
            - 'user_encoded2user': reverse mapping from encoded indices back to actual user IDs.

    Returns:
    --------
    DataFrame
        A DataFrame of similar animes with their similarity scores, sorted in descending order.
        Returns an empty DataFrame if the anime is not found or an error occurs.
    """

    # Get the anime ID from the name using helper function
    anime_id = get_anime_id_from_name(name)

    if anime_id is None:
        return

    # Get the encoded index of the anime
    encoded_index = encoded_dictionary['anime2anime_encoded'].get(anime_id)

    # normalize the embeddings, so cosine similarity with other
    # anime embeddings will simply be dot product
    anime_weights_norm = anime_weights/np.linalg.norm(anime_weights, axis = 1).reshape((-1, 1))
    dists = np.dot(anime_weights_norm, anime_weights_norm[encoded_index])
    dists_df = pd.DataFrame(dists, columns=['similarity'])

    # Store encoded index and map back to original anime IDs
    dists_df['anime_index'] = dists_df.index
    dists_df['anime_id'] = dists_df['anime_index'].map(encoded_dictionary['anime_encoded2anime'])

    # Remove the input anime itself from the results
    dists_df = dists_df[dists_df['anime_id'] != anime_id]

    # Sort by similarity in descending order
    dists_df_recommendations = dists_df.sort_values(by='similarity', ascending=False)

    return dists_df_recommendations


In [64]:
recommendations = model_rec_based_on_anime_similarity('Monster', anime_weights, encoded_dictionary)

In [65]:
filter_recommendations(recommendations, anime, anime_agg, Genres_preferred = ['Psychological','Thriller']).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,3002,Gyakkyou Burai Kaiji: Ultimate Survivor,Kaiji:Ultimate Survivor,Game|Psychological|Seinen|Thriller,TV,2007,8.261937,592
1,10271,Gyakkyou Burai Kaiji: Hakairoku-hen,Kaiji:Against All Rules,Game|Psychological|Seinen|Thriller,TV,2011,8.237178,1059
2,1520,Black Jack,Black Jack,Drama|Mystery|Psychological|Thriller,OVA,1993,7.804357,2918
3,7193,Aoi Bungaku Series,Unknown,Drama|Historical|Psychological|Seinen|Thriller,TV,2009,7.784752,882
4,1535,Death Note,Death Note,Mystery|Police|Psychological|Shounen|Supernatu...,TV,2006,8.632292,1
5,1943,Paprika,Paprika,Dementia|Fantasy|Horror|Mystery|Psychological|...,Movie,2006,8.013456,361
6,323,Mousou Dairinin,Paranoia Agent,Dementia|Drama|Mystery|Police|Psychological|Su...,TV,2004,7.603142,489
7,658,Touhai Densetsu Akagi: Yami ni Maiorita Tensai,Unknown,Game|Psychological|Seinen|Thriller,TV,2005,7.906526,1620
8,1889,Higurashi no Naku Koro ni Kai,When They Cry:Kai,Mystery|Psychological|Supernatural|Thriller,TV,2007,8.367541,344
9,37779,Yakusoku no Neverland,The Promised Neverland,Horror|Mystery|Psychological|Sci-Fi|Shounen|Th...,TV,2019,8.534097,56


In [66]:
recommendations = model_rec_based_on_anime_similarity('Naruto', anime_weights, encoded_dictionary)
filter_recommendations(recommendations, anime, anime_agg, Type_preferred = ['TV', 'Movie'], Genres_preferred = ['Psychological','Thriller']).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,1535,Death Note,Death Note,Mystery|Police|Psychological|Shounen|Supernatu...,TV,2006,8.632292,1
1,31240,Re:Zero kara Hajimeru Isekai Seikatsu,Re:ZERO -Starting Life in Another World-,Drama|Fantasy|Psychological|Thriller,TV,2016,8.249641,21
2,37779,Yakusoku no Neverland,The Promised Neverland,Horror|Mystery|Psychological|Sci-Fi|Shounen|Th...,TV,2019,8.534097,56
3,39617,Yakusoku no Neverland 2nd Season,The Promised Neverland Season 2,Mystery|Psychological|Sci-Fi|Shounen|Thriller,TV,2021,6.721898,249
4,37525,Babylon,Unknown,Mystery|Psychological|Thriller,TV,2019,6.630498,1354
5,39587,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,Re:ZERO -Starting Life in Another World- Season 2,Drama|Fantasy|Psychological|Thriller,TV,2020,8.405256,170
6,23283,Zankyou no Terror,Terror in Resonance,Mystery|Psychological|Thriller,TV,2014,8.056075,103
7,42203,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Seas...,Unknown,Drama|Fantasy|Psychological|Thriller,TV,2021,8.532015,402
8,36124,Itou Junji: Collection,The Junji Ito Collection,Comedy|Demons|Drama|Horror|Mystery|Psychologic...,TV,2018,5.949585,1449
9,10271,Gyakkyou Burai Kaiji: Hakairoku-hen,Kaiji:Against All Rules,Game|Psychological|Seinen|Thriller,TV,2011,8.237178,1059


### 2.Genre Based Similarity feature


#### 2.1 Recommending anime based on similarity in genres

In [67]:
anime_genres_mlb.head()

,MAL_ID,Genres_edited,Comedy,Action,Fantasy,Drama,Shounen,Adventure,Sci-Fi,Romance,...,Game,Police,Ai,Vampire,Samurai,Thriller,Josei,Dementia,Cars,Yaoi
0,1,Action|Adventure|Comedy|Drama|Sci-Fi|Space,1,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5,Action|Drama|Mystery|Sci-Fi|Space,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,6,Action|Adventure|Comedy|Drama|Sci-Fi|Shounen,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,7,Action|Drama|Magic|Mystery|Police|Supernatural,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,8,Adventure|Fantasy|Shounen|Supernatural,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
def rec_based_on_genre_similarity(anime_name, anime, anime_genre_mlb):
    """
    Recommends anime based on genre similarity to a given anime.

    Parameters:
    -----------
    anime_name : str
        The name of the anime for which recommendations are to be generated.
    anime : pd.DataFrame
    anime_genre_mlb : pd.DataFrame
        DataFrame with one-hot encoded genres using MultiLabelBinarizer.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing anime IDs and their similarity scores to the selected anime,
        sorted by similarity in descending order.
    """

    # Get the unique MAL_ID for the given anime name
    anime_id = get_anime_id_from_name(anime_name)

    if anime_id is None:
        return

    # Get the one-hot encoded genre vector for the selected anime
    selected_anime_genre_array = np.array(
        anime_genre_mlb.loc[anime_genre_mlb['MAL_ID'] == anime_id, anime_genre_mlb.columns[2:]]
    ).reshape(-1, 1)

    # Get the genre vectors of all animes
    anime_genre_array = anime_genre_mlb.loc[:, anime_genre_mlb.columns[2:]].values

    # Compute cosine similarity: dot product normalized by norms
    similarity = np.dot(anime_genre_array, selected_anime_genre_array).reshape(-1)
    normalization_factor = norm(anime_genre_array, axis=1) * norm(selected_anime_genre_array)

    # Avoid division by zero
    normalization_factor = np.where(normalization_factor == 0, 1, normalization_factor)

    # Final cosine similarity score
    similarity = similarity / normalization_factor

    # Prepare the output DataFrame
    similarity_df = pd.DataFrame(similarity, columns=['similarity'])
    similarity_df['MAL_ID'] = anime_genre_mlb['MAL_ID']

    # Remove the original anime from the recommendation list
    similarity_df = similarity_df[similarity_df['MAL_ID'] != anime_id]

    # Sort by similarity score in descending order
    similarity_df = similarity_df.sort_values(by='similarity', ascending=False)

    return similarity_df


In [69]:
anime.loc[anime['Name'] == 'Kakegurui', 'Genres_edited']

,Genres_edited
5360,Drama|Game|Mystery|Psychological|School|Shounen


In [70]:
filter_recommendations(rec_based_on_genre_similarity('Kakegurui', anime, anime_genres_mlb), anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,37086,Kakegurui××,Unknown,Drama|Game|Mystery|Psychological|School|Shounen,TV,2019,7.198555,304
1,14353,Death Billiards,Death Billiards,Drama|Game|Mystery|Psychological|Thriller,Movie,2013,7.880905,774
2,16201,Aku no Hana,Flowers of Evil,Drama|Psychological|Romance|School|Shounen,TV,2013,6.766187,909
3,28223,Death Parade,Death Parade,Drama|Game|Mystery|Psychological|Thriller,TV,2015,8.185751,47
4,341,Spiral: Suiri no Kizuna,Spiral:Bond of Reasoning,Drama|Mystery|Shounen,TV,2002,7.334778,2307
5,21557,Omoide no Marnie,When Marnie Was There,Drama|Mystery|Psychological,Movie,2014,8.120706,784
6,28851,Koe no Katachi,A Silent Voice,Drama|School|Shounen,Movie,2016,8.944652,28
7,252,Madlax,Unknown,Drama|Magic|Military|Mystery|Psychological|Sho...,TV,2004,7.079229,2672
8,14075,Zetsuen no Tempest,Blast of Tempest,Action|Drama|Fantasy|Magic|Mystery|Psychologic...,TV,2012,7.986864,247
9,33377,"Trickster: Edogawa Ranpo ""Shounen Tanteidan"" yori",Trickster,Drama|Mystery|Psychological|Sci-Fi,TV,2016,5.880952,1407


In [71]:
import gc
gc.collect()

509

#### 2.2 Combining Genre based Similarity and Anime-Anime Similarity via Embeddings


In [72]:
def rec_based_on_comb_of_genre_sim_and_model(anime_name, anime_weights, encoded_dictionary, anime, anime_genres_mlb,
                                             threshold_genre_similarity=0.5, threshold_model_anime_similarity=0.3):
    """
    Recommend anime by combining genre-based and model-based (embedding) similarity.

    Parameters:
    -----------
    anime_name : str
        Name of the anime to find similar titles for.
    anime_weights : np.ndarray
        Embedding matrix of anime from the trained deep learning model.
    encoded_dictionary : dict
        Dictionary containing:
            - 'anime2anime_encoded': mapping from actual anime IDs to encoded indices.
            - 'anime_encoded2anime': reverse mapping from encoded indices back to actual anime IDs.
            - 'user2user_encoded': mapping from actual user IDs to encoded indices.
            - 'user_encoded2user': reverse mapping from encoded indices back to actual user IDs.
    anime : pd.DataFrame
        DataFrame containing anime metadata including 'MAL_ID', 'Name', etc.
    anime_genres_mlb : pd.DataFrame
        One-hot encoded genre vectors using MultiLabelBinarizer.
    threshold_genre_similarity : float, optional
        Minimum genre similarity to be considered in recommendations.
    threshold_model_anime_similarity : float, optional
        Minimum model-based anime similarity to be considered in recommendations.

    Returns:
    --------
    pd.DataFrame
        DataFrame of recommended anime sorted by model similarity.
    """

    # Get recommendations based on model's anime-anime similarity (dot product of embeddings)
    rec_based_on_model_df = model_rec_based_on_anime_similarity(anime_name, anime_weights, encoded_dictionary)

    # Get recommendations based on genre similarity (cosine similarity of genre vectors)
    rec_based_on_genre_similarity_df = rec_based_on_genre_similarity(anime_name, anime, anime_genres_mlb)

    if rec_based_on_model_df is None or rec_based_on_genre_similarity_df is None:
        return

    # Merge the two recommendations on anime ID to retain only overlapping results
    comb_rec = rec_based_on_genre_similarity_df[['similarity', 'MAL_ID']].merge(
        rec_based_on_model_df[['similarity', 'anime_id']],
        left_on='MAL_ID',
        right_on='anime_id',
        how='inner'
    )

    # Drop the duplicate anime_id column and rename similarity columns for clarity
    comb_rec = comb_rec.drop('anime_id', axis=1)
    comb_rec = comb_rec.rename(columns={
        'similarity_x': 'similarity_genre',
        'similarity_y': 'similarity_model'
    })

    # Filter based on both genre and model similarity thresholds
    comb_rec = comb_rec[
        (comb_rec['similarity_genre'] >= threshold_genre_similarity) &
        (comb_rec['similarity_model'] >= threshold_model_anime_similarity)
    ]

    # Sort by model similarity
    comb_rec = comb_rec.sort_values(by='similarity_model', ascending=False)

    return comb_rec


In [73]:
filter_recommendations(rec_based_on_comb_of_genre_sim_and_model('Kaiji:Ultimate Survivor', anime_weights, encoded_dictionary, anime, anime_genres_mlb), anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,10271,Gyakkyou Burai Kaiji: Hakairoku-hen,Kaiji:Against All Rules,Game|Psychological|Seinen|Thriller,TV,2011,8.237178,1059
1,19,Monster,Monster,Drama|Horror|Mystery|Police|Psychological|Sein...,TV,2004,8.722994,169
2,5040,One Outs,One Outs,Psychological|Seinen|Sports,TV,2008,8.341930,815
3,6114,Rainbow: Nisha Rokubou no Shichinin,Rainbow,Drama|Historical|Seinen|Thriller,TV,2010,8.445601,466
4,658,Touhai Densetsu Akagi: Yami ni Maiorita Tensai,Unknown,Game|Psychological|Seinen|Thriller,TV,2005,7.906526,1620
5,22135,Ping Pong the Animation,Ping Pong the Animation,Drama|Psychological|Seinen|Sports,TV,2014,8.494295,543
6,1520,Black Jack,Black Jack,Drama|Mystery|Psychological|Thriller,OVA,1993,7.804357,2918
7,6774,Kuuchuu Buranko,Welcome to Irabu's Office,Comedy|Drama|Psychological|Seinen,TV,2009,7.866792,1753
8,35180,3-gatsu no Lion 2nd Season,March Comes In Like A Lion 2nd Season,Drama|Game|Seinen|Slice-of-Life,TV,2017,9.007975,544
9,7193,Aoi Bungaku Series,Unknown,Drama|Historical|Psychological|Seinen|Thriller,TV,2009,7.784752,882


### 3.Content based similarity feature

In [74]:
def content_similarity(anime_index, content_df, embeddings_column_name, anime_id):

    try:
        user_anime_vector = content_df.loc[anime_index, embeddings_column_name]
        results = pd.DataFrame(content_df[embeddings_column_name].apply(lambda x: util.pytorch_cos_sim(user_anime_vector, x).item()))
        results.columns = ['similarity_score']
        results['MAL_ID'] = content_df['MAL_ID']
        results = results[results['MAL_ID'] != anime_id]

    except Exception as e:
        print(e)
        return

    return results

In [75]:
def content_based_rec(anime_name, content_df, anime):
    """
    Recommends anime based on plot similarity using content embeddings.

    Parameters:
    -----------
    anime_name : str
        Name of the anime for which similar anime are to be recommended.
    content_df : pd.DataFrame
        DataFrame containing anime information and their content embeddings
    anime : pd.DataFrame
        DataFrame containing full anime metadata including MAL_ID and Name.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing anime recommendations ranked by similarity score.
    """

    # Convert anime name to MAL_ID
    anime_id = get_anime_id_from_name(anime_name)

    if anime_id is None:
        return

    # Get the index of the anime in the content dataframe
    anime_index = content_df[content_df['MAL_ID'] == anime_id].index[0]

    # Calculate similarity with other anime using model embeddings
    results = content_similarity(anime_index, content_df, 'plot_embeddings', anime_id)

    # Sort the results by similarity_score
    results = results.sort_values(by = 'similarity_score', ascending = False)

    return results


In [76]:
filter_recommendations(content_based_rec('One Outs', content_df, anime), anime, anime_agg)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,386,H2,Unknown,Comedy|Romance|Shounen|Sports,TV,1995,7.267803,5007
1,5941,Cross Game,Cross Game,Comedy|Drama|Romance|School|Shounen|Sports,TV,2009,8.458564,1386
2,18689,Diamond no Ace,Ace of Diamond,Comedy|School|Shounen|Sports,TV,2013,8.060439,826
3,1614,Captain Tsubasa: Road to 2002,Captain Tsubasa,Action|Shounen|Sports,TV,2001,7.220598,2466
4,2159,Ookiku Furikabutte,Big Windup!,Comedy|Seinen|Sports,TV,2007,7.979758,1609
...,...,...,...,...,...,...,...,...
5138,40128,Arte,Arte,Drama|Historical|Romance|Seinen|Slice-of-Life,TV,2020,7.005959,1647
5139,345,Eikoku Koi Monogatari Emma,Emma:A Victorian Romance,Drama|Historical|Romance|Seinen|Slice-of-Life,TV,2005,7.692542,2397
5140,3829,Anne no Nikki,The Diary of Anne Frank,Drama|Historical,Movie,1995,6.870504,5339
5141,13375,Kyoto Animation: Sora-hen,Sky,Slice-of-Life,Special,2010,5.581960,6037


In [77]:
filter_recommendations(content_based_rec('Gyakkyou Burai Kaiji: Ultimate Survivor', content_df, anime), anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,10271,Gyakkyou Burai Kaiji: Hakairoku-hen,Kaiji:Against All Rules,Game|Psychological|Seinen|Thriller,TV,2011,8.237178,1059
1,34933,Kakegurui,Kakegurui:Compulsive Gambler,Drama|Game|Mystery|Psychological|School|Shounen,TV,2017,7.288598,97
2,10163,C: The Money of Soul and Possibility Control,[C] CONTROL - The Money and Soul of Possibility,Action|Mystery|Super-Power|Thriller,TV,2011,7.246433,581
3,40052,Great Pretender,Unknown,Action|Adventure|Comedy|Mystery|Psychological,ONA,2020,8.262616,520
4,30016,Nanbaka,Nanbaka,Action|Comedy|Drama,TV,2016,7.039210,619
5,33197,Lostorage Incited WIXOSS,Lostorage incited WIXOSS,Game|Psychological,TV,2016,6.582844,2268
6,1193,Happy Seven: The TV Manga,Unknown,Comedy|Fantasy|Magic|School,TV,2005,6.330619,5777
7,3091,xxxHOLiC Kei,Unknown,Comedy|Drama|Mystery|Psychological|Supernatural,TV,2008,8.340262,1240
8,36027,Ousama Game The Animation,King's Game,Dementia|Drama|Horror|Mystery|School|Supernatural,TV,2017,4.632233,844
9,3369,Shoubushi Densetsu Tetsuya,Unknown,Game|Historical|Shounen,TV,2000,7.601911,5152


### 4.Recommendation Based on User's Taste in Different Genres

In [78]:
user_agg.head()

,user_id,animes_watched,user_avg_rating
0,0,40,7.725000
1,1,117,8.051282
2,2,46,8.521739
3,3,317,7.611987
4,4,120,7.633333


In [79]:
def adding_genre_info(user, anime, genre_list, user_agg):
    """
    Enhances the user-level aggregate dataframe by adding genre-specific information

    Parameters:
    - user (DataFrame): User-anime rating data
    - anime (DataFrame): Anime metadata
    - genre_list (list): List of genres to compute statistics for.
    - user_agg (DataFrame): Initial user-level aggregate dataframe.

    Returns:
    - user_agg_with_genre (DataFrame): The input dataframe enhanced with genre-wise features.
    """
    user_agg_with_genre = user_agg.copy()

    for genre_chosen in genre_list:
        # Filter anime that contain the current genre
        genre_animes_cond = anime['Genres_edited'].str.split('|').apply(lambda x: genre_chosen in x)
        genre_animes = anime[genre_animes_cond]['MAL_ID'].tolist()

        # For each user, calculate:
        # - count: number of anime watched in this genre
        # - mean: average rating_diff for anime in this genre
        # - std: standard deviation of rating_diff
        genre_user = user[user['anime_id'].isin(genre_animes)].groupby('user_id')['rating_diff'].agg(['count', 'mean', 'std'])

        # Fill missing standard deviations (users with only 1 anime) with 1 to avoid division by 0
        genre_user['std'] = genre_user['std'].fillna(1)

        # Adjust the mean by dividing with std dev (or a floor of 0.8 to avoid exaggeration)
        floor_std_dev = 0.8
        genre_user['adjusted_mean'] = genre_user['mean'] / genre_user['std'].apply(lambda x: max(x, floor_std_dev))

        # Clean up and rename for clarity
        genre_user = genre_user.drop(['mean', 'std'], axis=1).reset_index()
        genre_user = genre_user.rename(columns={
            "count": genre_chosen + '_animes_watched',
            "adjusted_mean": genre_chosen + '_avg_rating_diff'
        })

        # Merge the genre-specific features back to user aggregate dataframe
        user_agg_with_genre = pd.merge(user_agg_with_genre, genre_user, on='user_id', how='left')

        # Free memory after each genre loop
        gc.collect()

    return user_agg_with_genre


In [80]:
user_agg_with_genres = adding_genre_info(user, anime, genres_list, user_agg)
user_agg_with_genres.head()

,user_id,animes_watched,user_avg_rating,Comedy_animes_watched,Comedy_avg_rating_diff,Action_animes_watched,Action_avg_rating_diff,Fantasy_animes_watched,Fantasy_avg_rating_diff,Drama_animes_watched,...,Thriller_animes_watched,Thriller_avg_rating_diff,Josei_animes_watched,Josei_avg_rating_diff,Dementia_animes_watched,Dementia_avg_rating_diff,Cars_animes_watched,Cars_avg_rating_diff,Yaoi_animes_watched,Yaoi_avg_rating_diff
0,0,40,7.725000,21.0,0.251836,12.0,0.106664,14.0,0.343056,15.0,...,NaN,NaN,1.0,2.448345,NaN,NaN,NaN,NaN,NaN,NaN
1,1,117,8.051282,55.0,-0.169704,53.0,-0.035626,27.0,-0.070773,44.0,...,9.0,1.167890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,46,8.521739,20.0,0.642896,30.0,0.416501,4.0,0.179347,15.0,...,4.0,0.380513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,317,7.611987,193.0,-0.030039,119.0,-0.028984,49.0,0.128290,71.0,...,16.0,0.257884,1.0,0.449025,3.0,0.22984,8.0,0.248089,NaN,NaN
4,4,120,7.633333,56.0,0.070432,33.0,-0.021941,40.0,0.017039,70.0,...,3.0,0.359146,6.0,0.449656,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# user_agg_with_genres.to_pickle('datasets/manual_save/user_agg_with_genres.pkl')

In [ ]:
# user_agg_with_genres = pd.read_pickle('datasets/manual_save/user_agg_with_genres.pkl')

In [81]:
user_agg_for_ind_genre_preference = user_agg_with_genres.copy()

In [82]:
user_agg_for_ind_genre_preference['max_animes_watched_of_a_genre'] = user_agg_for_ind_genre_preference[[column for column in user_agg_for_ind_genre_preference.columns if '_animes_watched' in column ]].max(axis = 1)
user_agg_for_ind_genre_preference['max_animes_watched_of_a_genre']

,max_animes_watched_of_a_genre
0,21.0
1,65.0
2,30.0
3,193.0
4,74.0
...,...
312115,46.0
312116,45.0
312117,23.0
312118,40.0


In [83]:
for genre in genres_list:
    user_agg_for_ind_genre_preference.loc[user_agg_for_ind_genre_preference[genre + '_animes_watched'] < user_agg_for_ind_genre_preference['max_animes_watched_of_a_genre'].apply(lambda x: max(8, x**0.5)), genre + '_avg_rating_diff'] = np.nan
    user_agg_for_ind_genre_preference.loc[user_agg_for_ind_genre_preference[genre + '_animes_watched'] < user_agg_for_ind_genre_preference['max_animes_watched_of_a_genre'].apply(lambda x: max(8, x**0.5)), genre + '_animes_watched'] = np.nan

In [84]:
user_agg_for_ind_genre_preference.head()

,user_id,animes_watched,user_avg_rating,Comedy_animes_watched,Comedy_avg_rating_diff,Action_animes_watched,Action_avg_rating_diff,Fantasy_animes_watched,Fantasy_avg_rating_diff,Drama_animes_watched,...,Thriller_avg_rating_diff,Josei_animes_watched,Josei_avg_rating_diff,Dementia_animes_watched,Dementia_avg_rating_diff,Cars_animes_watched,Cars_avg_rating_diff,Yaoi_animes_watched,Yaoi_avg_rating_diff,max_animes_watched_of_a_genre
0,0,40,7.725000,21.0,0.251836,12.0,0.106664,14.0,0.343056,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
1,1,117,8.051282,55.0,-0.169704,53.0,-0.035626,27.0,-0.070773,44.0,...,1.167890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0
2,2,46,8.521739,20.0,0.642896,30.0,0.416501,NaN,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
3,3,317,7.611987,193.0,-0.030039,119.0,-0.028984,49.0,0.128290,71.0,...,0.257884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193.0
4,4,120,7.633333,56.0,0.070432,33.0,-0.021941,40.0,0.017039,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0


In [85]:
def anime_rec_based_on_ind_genre_preference(user_agg_for_ind_genre_preference, user_id, user, anime_agg, anime):
    """
    Recommends anime to a user based on their individual genre preferences derived from adjusted mean rating differences.

    Parameters:
    - user_agg_for_ind_genre_preference (DataFrame): User-level data with genre-wise rating diffs and anime counts.
    - user_id (int): The ID of the user for whom we want recommendations.
    - user (DataFrame): User-anime rating data
    - anime_agg (DataFrame): Aggregated anime data
    - anime (DataFrame): Original anime metadata.

    Returns:
    - DataFrame: A list of anime sorted by genre preference.
                Returns None if user doesn't have enough valid genre preferences.
    """

    # Get all columns that contain genre-based info
    avg_rating_diff_columns = [col for col in user_agg_for_ind_genre_preference.columns if col.endswith('_avg_rating_diff')]
    count_columns = [col for col in user_agg_for_ind_genre_preference.columns if col.endswith('_animes_watched')]

    # Extract user's data
    user_genre_preference = user_agg_for_ind_genre_preference.loc[
        user_agg_for_ind_genre_preference['user_id'] == user_id, avg_rating_diff_columns]
    user_genre_preference_count = user_agg_for_ind_genre_preference.loc[
        user_agg_for_ind_genre_preference['user_id'] == user_id, count_columns]

    # Keep only genres where user has valid (non-NaN) data
    valid_avg_rating_diff_columns = user_genre_preference.columns[~user_genre_preference.isna().any()].tolist()
    valid_count_columns = user_genre_preference_count.columns[~user_genre_preference_count.isna().any()].tolist()
    valid_genres = [col.removesuffix('_avg_rating_diff') for col in valid_avg_rating_diff_columns]

    # Require the user to have at least 5 genres with sufficient data to make a meaningful profile
    if len(valid_avg_rating_diff_columns) < 5:
        print('Not enough Animes watched')
        return None

    # Extract the genre weights and counts
    user_genre_weights = user_genre_preference[valid_avg_rating_diff_columns].values[0]
    user_genre_weights_count = user_genre_preference_count[valid_count_columns].values[0]

    # Normalize genre weights between [0.1, 1] using min-max scaling
    interval_min = 0.1
    interval_max = 1
    user_genre_weights_norm = ((user_genre_weights - np.min(user_genre_weights)) /
                               (np.max(user_genre_weights) - np.min(user_genre_weights))) * \
                              (interval_max - interval_min) + interval_min

    # Compute preference score for each anime based on user's genre weight vector
    user_anime_preference_based_on_genre = anime_genres_mlb.set_index('MAL_ID')[valid_genres].dot(user_genre_weights_norm)

    # Normalize the score using a soft penalty for too many genres per anime (to avoid bias toward all-genre anime)
    dividing_factor = (anime_genres_mlb.set_index('MAL_ID').drop(['Genres_edited'], axis=1).sum(axis=1)).pow(0.88)
    user_anime_preference_based_on_genre = user_anime_preference_based_on_genre / dividing_factor
    user_anime_preference_based_on_genre = user_anime_preference_based_on_genre[user_anime_preference_based_on_genre.notna()]

    # Exclude anime already watched by the user
    animes_watched_by_user = user[user['user_id'] == user_id]['anime_id'].values.tolist()
    user_anime_preference_based_on_genre = user_anime_preference_based_on_genre[~user_anime_preference_based_on_genre.index.isin(animes_watched_by_user)]

    # Prepare final DataFrame
    user_anime_rec_based_on_ind_genre_preference = user_anime_preference_based_on_genre.sort_values(ascending=False).reset_index()
    user_anime_rec_based_on_ind_genre_preference.columns = ['MAL_ID', 'preference_score']

    # Add average rating for sorting purposes
    user_anime_rec_based_on_ind_genre_preference['anime_avg_rating'] = \
        user_anime_rec_based_on_ind_genre_preference['MAL_ID'].map(anime_agg.set_index('anime_id')['anime_avg_rating'])

    # Sort by preference score and then average rating
    user_anime_rec_based_on_ind_genre_preference.sort_values(by=['preference_score', 'anime_avg_rating'], ascending=[False, False], inplace=True)

    return user_anime_rec_based_on_ind_genre_preference


In [86]:
user_anime_rec_based_on_ind_genre_preference = anime_rec_based_on_ind_genre_preference(user_agg_for_ind_genre_preference, get_random_user_id(), user, anime_agg, anime)

In [87]:
filter_recommendations(user_anime_rec_based_on_ind_genre_preference, anime, anime_agg, min_anime_rating= 7.5, Type_preferred=['TV']).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,40815,Honzuki no Gekokujou: Shisho ni Naru Tame ni w...,Ascendance of a Bookworm Part 2,Fantasy|Slice-of-Life,TV,2020,8.137708,1395
1,39468,Honzuki no Gekokujou: Shisho ni Naru Tame ni w...,Ascendance of a Bookworm,Fantasy|Slice-of-Life,TV,2019,7.961125,994
2,33352,Violet Evergarden,Violet Evergarden,Drama|Fantasy|Slice-of-Life,TV,2018,8.480014,67
3,5420,Kemono no Souja Erin,Erin,Drama|Fantasy|Slice-of-Life,TV,2009,8.306033,1815
4,35790,Tate no Yuusha no Nariagari,The Rising of the Shield Hero,Action|Adventure|Drama|Fantasy,TV,2019,7.871677,85
5,31859,Hai to Gensou no Grimgar,Grimgar:Ashes and Illusions,Action|Adventure|Drama|Fantasy,TV,2016,7.570912,215
6,18153,Kyoukai no Kanata,Beyond the Boundary,Fantasy|Slice-of-Life|Supernatural,TV,2013,7.638018,100
7,39575,Somali to Mori no Kamisama,Somali and the Forest Spirit,Adventure|Demons|Drama|Fantasy|Slice-of-Life,TV,2020,7.680228,772
8,16067,Nagi no Asu kara,A Lull in the Sea,Drama|Fantasy|Romance,TV,2013,8.069194,268
9,33206,Kobayashi-san Chi no Maid Dragon,Miss Kobayashi's Dragon Maid,Comedy|Fantasy|Slice-of-Life,TV,2017,8.014112,109


### 5.Recommendation based on Genre-Based User Similarity

We identify users with similar genre preferences using two methods (two features): \
1. Watch Count-Based Similarity
2. Rating-Based Similarity

In [88]:
def return_user_agg_for_user_user_genre_rating_preference(user_agg_with_genres, genres_list):

    user_agg_for_user_user_genre_rating_preference =  user_agg_with_genres[user_agg_with_genres['animes_watched'] >= 50]

    for genre in genres_list:
        user_agg_for_user_user_genre_rating_preference.loc[user_agg_for_user_user_genre_rating_preference[genre + '_animes_watched'] < 5, genre + '_avg_rating_diff'] = np.nan
        user_agg_for_user_user_genre_rating_preference.loc[user_agg_for_user_user_genre_rating_preference[genre + '_animes_watched'] < 5, genre + '_animes_watched'] = np.nan

    return user_agg_for_user_user_genre_rating_preference

In [89]:
def return_user_agg_for_user_user_genre_watch_preference(user_agg_with_genres, genres_list):

    user_agg_for_user_user_genre_watch_preference =  user_agg_with_genres[user_agg_with_genres['animes_watched'] >= 50]
    user_agg_for_user_user_genre_watch_preference = user_agg_for_user_user_genre_watch_preference.fillna(0)

    return user_agg_for_user_user_genre_watch_preference

In [90]:
user_agg_for_user_user_genre_rating_preference = return_user_agg_for_user_user_genre_rating_preference(user_agg_with_genres, genres_list)
user_agg_for_user_user_genre_watch_preference  = return_user_agg_for_user_user_genre_watch_preference(user_agg_with_genres, genres_list)

In [91]:
def rec_based_on_user_user_genre_preference(user_agg_for_user_user_genre_rating_preference,
                                            user_agg_for_user_user_genre_watch_preference,
                                            genres_list, user, user_id,
                                            preference_type='watch',
                                            top_similar_users=100,
                                            min_fraction_of_top_similar_users_for_anime_rec=0.2,
                                            rec_method_by='users_watched'):
    """
    Recommend anime to a user by finding similar users based on genre preferences
    (either based on watch count or rating differences).

    Parameters:
    - user_agg_for_user_user_genre_rating_preference (DataFrame): User-genre aggregated rating diffs.
    - user_agg_for_user_user_genre_watch_preference (DataFrame): User-genre aggregated watch counts.
    - genres_list (list): List of genres considered.
    - user (DataFrame): Original user-anime rating data.
    - user_id (int): ID of the user to generate recommendations for.
    - preference_type (str): 'watch' or 'rating' — how to measure genre preference similarity.
    - top_similar_users (int): Number of most similar users to consider for recommendations.
    - min_fraction_of_top_similar_users_for_anime_rec (float): Minimum fraction of similar users who must have watched an anime for it to be considered.
    - rec_method_by (str): Metric to sort recommendations by. One of ['users_watched', 'rating_mean', 'rating_diff_mean'].

    Returns:
    - DataFrame: Anime recommendations sorted by the specified method, One of ['users_watched', 'rating_mean', 'rating_diff_mean'].
    """

    # Select the correct user-genre preference dataframe and relevant columns based on type
    if preference_type == 'rating':
        user_agg_for_user_user_genre_preference = user_agg_for_user_user_genre_rating_preference
        if user_id not in user_agg_for_user_user_genre_preference['user_id'].values:
            print('Not enough Animes watched')
            return None
        columns = [genre + '_avg_rating_diff' for genre in genres_list]

        # We will consider only non NA rating columns of the user
        rel_columns = user_agg_for_user_user_genre_preference.loc[
            user_agg_for_user_user_genre_preference['user_id'] == user_id, columns] \
            .apply(lambda row: row[pd.notnull(row)].index.tolist(), axis=1).values[0]

        user_agg_for_user_user_genre_preference = user_agg_for_user_user_genre_preference.dropna(subset = rel_columns)

    elif preference_type == 'watch':
        user_agg_for_user_user_genre_preference = user_agg_for_user_user_genre_watch_preference
        if user_id not in user_agg_for_user_user_genre_preference['user_id'].values:
            print('Not enough Animes watched')
            return None
        rel_columns = [genre + '_animes_watched' for genre in genres_list]

    # Extract the genre vector for the target user
    user_genre_scores_ind = np.array(
        user_agg_for_user_user_genre_preference.loc[
            user_agg_for_user_user_genre_preference['user_id'] == user_id, rel_columns
        ]
    ).reshape(1, -1)

    # Extract genre vectors for all users
    user_genre_scores_all = user_agg_for_user_user_genre_preference.loc[:, rel_columns].values

    # Compute pairwise Euclidean distance between the target user and all others
    genre_distance_array = distance.cdist(user_genre_scores_all, user_genre_scores_ind, metric='euclidean')

    # Construct a dataframe of distances
    genre_distance_df = user_agg_for_user_user_genre_preference[['user_id']].copy()
    genre_distance_df['genre_euclidean_distance'] = genre_distance_array.flatten()
    genre_distance_df.sort_values(by='genre_euclidean_distance', ascending = True, inplace=True)

    # Exclude the target user from similar users
    genre_distance_df = genre_distance_df[genre_distance_df['user_id'] != user_id]

    # Select top N most similar users
    genre_distance_df = genre_distance_df.iloc[:top_similar_users]

    # Get anime watched by those similar users
    user_user_genre_rec = user[user['user_id'].isin(genre_distance_df['user_id'])]

    # Exclude anime already watched by the target user
    animes_watched_by_user = user[user['user_id'] == user_id]['anime_id'].unique().tolist()
    user_user_genre_rec = user_user_genre_rec[~user_user_genre_rec['anime_id'].isin(animes_watched_by_user)]

    # Group by anime to compute stats for recommendation scoring
    user_user_genre_rec_grouped = user_user_genre_rec.groupby('anime_id').agg(
        users_watched=('rating', 'count'),
        rating_mean=('rating', lambda x: stats.trim_mean(x, 0.05)),  # Trimmed mean to reduce outlier impact
        rating_diff_mean=('rating_diff', lambda y: stats.trim_mean(y, 0.05))
    ).reset_index()

    # Filter out anime watched by too few similar users
    min_users_watched = min_fraction_of_top_similar_users_for_anime_rec * top_similar_users
    user_user_genre_rec_grouped = user_user_genre_rec_grouped[
        user_user_genre_rec_grouped['users_watched'] >= min_users_watched
    ]

    # Final sorting by selected metric (e.g., 'users_watched', 'rating_mean', etc.)
    user_user_genre_rec_grouped = user_user_genre_rec_grouped.sort_values(by=rec_method_by, ascending=False)

    return user_user_genre_rec_grouped


In [92]:
selected_user_id_3 = get_random_user_id()
selected_user_id_3

np.int64(258543)

In [93]:
user_user_genre_preference_rec = rec_based_on_user_user_genre_preference(
                                    user_agg_for_user_user_genre_rating_preference,
                                    user_agg_for_user_user_genre_watch_preference,
                                    genres_list, user, selected_user_id_3,
                                    preference_type='watch',
                                    top_similar_users=100,
                                    min_fraction_of_top_similar_users_for_anime_rec=0.3,
                                    rec_method_by='rating_mean')

In [94]:
user_user_genre_preference_rec.head(10)

,anime_id,users_watched,rating_mean,rating_diff_mean
432,5114,47,9.558140,0.939874
131,431,37,9.142857,0.858322
1053,28851,32,9.133333,0.718431
73,199,51,9.106383,0.730422
717,12355,35,8.909091,0.511399
649,10408,42,8.868421,0.671458
1097,31043,36,8.823529,0.704457
420,4898,40,8.694444,0.690438
956,22319,47,8.604651,0.696276
495,6547,61,8.472727,0.331614


In [95]:
filter_recommendations(user_user_genre_preference_rec, anime, anime_agg, Type_preferred= ['TV']).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist:Brotherhood,Action|Adventure|Comedy|Drama|Fantasy|Magic|Mi...,TV,2009,9.148895,3
1,31043,Boku dake ga Inai Machi,ERASED,Mystery|Psychological|Seinen|Supernatural,TV,2016,8.301554,29
2,4898,Kuroshitsuji,Black Butler,Action|Comedy|Demons|Historical|Mystery|Shoune...,TV,2008,7.778306,93
3,22319,Tokyo Ghoul,Tokyo Ghoul,Action|Drama|Horror|Mystery|Psychological|Sein...,TV,2014,7.659684,7
4,6547,Angel Beats!,Angel Beats!,Action|Comedy|Drama|School|Supernatural,TV,2010,8.185892,14
5,9656,Kimi ni Todoke 2nd Season,Kimi ni Todoke:From Me To You 2,Drama|Romance|School|Shoujo|Slice-of-Life,TV,2011,8.085564,411
6,120,Fruits Basket,Fruits Basket,Comedy|Drama|Romance|Shoujo|Slice-of-Life|Supe...,TV,2001,7.747061,314
7,4752,Vampire Knight: Guilty,Vampire Knight:Guilty,Drama|Mystery|Romance|Shoujo|Supernatural|Vampire,TV,2008,7.366674,464
8,18507,Free!,Free! - Iwatobi Swim Club,Comedy|Drama|School|Slice-of-Life|Sports,TV,2013,7.359428,189
9,15809,Hataraku Maou-sama!,The Devil is a Part-Timer!,Comedy|Demons|Fantasy|Romance|Supernatural,TV,2013,7.843557,62


### 6.Watch history based Recommendation

In [96]:
def watch_history_based_rec(user_id, user_agg, user, anime,
                            min_common_ratio=0.2, top_similar_users=100,
                            rec_method_by='users_watched',
                            min_fraction_of_top_similar_users_for_anime_rec=0.2):
    """
    Recommend anime based on users with similar watch histories.

    Parameters:
    - user_id (int): ID of the target user.
    - user_agg (DataFrame): User-level aggregated stats
    - user (DataFrame): User-anime interactions
    - anime (DataFrame): Anime metadata
    - min_common_ratio (float): Minimum ratio of overlapping watched anime for similarity.
    - top_similar_users (int): Number of most similar users to consider.
    - rec_method_by (str): Metric to sort recommendations by ('users_watched', 'rating_mean', or 'rating_diff_mean').
    - min_fraction_of_top_similar_users_for_anime_rec (float): Threshold for minimum similar user engagement in a recommended anime.

    Returns:
    - DataFrame: Recommended anime sorted by the chosen metric.
    """

    # Get how many animes the target user has watched
    no_of_animes_watched_by_user = user_agg[user_agg['user_id'] == user_id]['animes_watched'].values[0]

    # Filter users who have watched a similar number of animes (within ~80%-125% range)
    selected_users_agg = user_agg[
        user_agg['animes_watched'].between(
            int(0.8 * no_of_animes_watched_by_user),
            int(1.25 * no_of_animes_watched_by_user)
        )
    ]
    selected_users_agg = selected_users_agg[selected_users_agg['user_id'] != user_id]

    # Get the list of animes the user already watched
    animes_already_watched = user[user['user_id'] == user_id]['anime_id'].unique().tolist()

    # Get interactions of selected users on those common animes
    selected_users_df = user[user['user_id'].isin(selected_users_agg['user_id'])]
    selected_users_df = selected_users_df[selected_users_df['anime_id'].isin(animes_already_watched)]

    # Count how many common animes each user has with the target user
    selected_users_common = selected_users_df['user_id'].value_counts().reset_index(name='common_animes')
    selected_users_common.rename(columns={'index': 'user_id'}, inplace=True)

    # Add total animes watched by each similar user
    selected_users_common['animes_watched'] = selected_users_common['user_id'].map(
        user_agg.set_index('user_id')['animes_watched'])

    # Compute the ratio of overlap
    selected_users_common['common_animes_ratio'] = (
        selected_users_common['common_animes'] / selected_users_common['animes_watched'])

    # Filter out users with low overlap
    selected_users_common = selected_users_common[
        selected_users_common['common_animes_ratio'] >= min_common_ratio]

    # Apply threshold to get top similar users by overlap
    thr_common_ratio = selected_users_common['common_animes_ratio'].sort_values(ascending=False).iloc[top_similar_users]
    selected_users_common = selected_users_common[
        selected_users_common['common_animes_ratio'] >= thr_common_ratio]

    # Get recommendations: animes watched by similar users that the target user hasn't seen
    rec_df = user[user['user_id'].isin(selected_users_common['user_id'])]
    rec_df = rec_df[~rec_df['anime_id'].isin(animes_already_watched)]

    # Aggregate recommendation stats
    rec_df_grouped = rec_df.groupby('anime_id').agg(
        users_watched=('rating', 'count'),
        rating_mean=('rating', lambda x: stats.trim_mean(x, 0.05)),
        rating_diff_mean=('rating_diff', lambda x: stats.trim_mean(x, 0.05))
    ).reset_index()

    # Filter anime watched by a minimum number of similar users
    min_users_watched = min_fraction_of_top_similar_users_for_anime_rec * top_similar_users
    rec_df_grouped = rec_df_grouped[rec_df_grouped['users_watched'] >= min_users_watched]

    # Sort by the chosen metric
    rec_df_grouped = rec_df_grouped.sort_values(by=rec_method_by, ascending=False)
    rec_df_grouped = rec_df_grouped.reset_index(drop = True)

    return rec_df_grouped


In [97]:
watch_history_based_recommendations = watch_history_based_rec(
                                      selected_user_id_3, user_agg, user, anime,
                                      min_common_ratio=0.2, top_similar_users=100,
                                      rec_method_by='users_watched',
                                      min_fraction_of_top_similar_users_for_anime_rec=0.2)
watch_history_based_recommendations.head(10)

,anime_id,users_watched,rating_mean,rating_diff_mean
0,22319,70,8.687500,0.773147
1,6547,62,8.267857,0.124898
2,11771,48,8.409091,0.245952
3,9656,48,8.272727,0.130675
4,21881,43,7.717949,0.231478
5,15809,42,8.078947,0.048433
6,27899,42,7.973684,0.389665
7,18507,40,7.777778,0.048572
8,22199,39,7.675676,-0.057604
9,5114,39,9.351351,0.772969


In [98]:
filter_recommendations(watch_history_based_recommendations, anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,22319,Tokyo Ghoul,Tokyo Ghoul,Action|Drama|Horror|Mystery|Psychological|Sein...,TV,2014,7.659684,7
1,6547,Angel Beats!,Angel Beats!,Action|Comedy|Drama|School|Supernatural,TV,2010,8.185892,14
2,11771,Kuroko no Basket,Kuroko's Basketball,Comedy|School|Shounen|Sports,TV,2012,8.191416,104
3,9656,Kimi ni Todoke 2nd Season,Kimi ni Todoke:From Me To You 2,Drama|Romance|School|Shoujo|Slice-of-Life,TV,2011,8.085564,411
4,21881,Sword Art Online II,Sword Art Online II,Action|Adventure|Fantasy|Game|Romance,TV,2014,6.854802,23
5,15809,Hataraku Maou-sama!,The Devil is a Part-Timer!,Comedy|Demons|Fantasy|Romance|Supernatural,TV,2013,7.843557,62
6,27899,Tokyo Ghoul √A,Tokyo Ghoul √A,Action|Drama|Horror|Mystery|Psychological|Sein...,TV,2015,6.912570,42
7,18507,Free!,Free! - Iwatobi Swim Club,Comedy|Drama|School|Slice-of-Life|Sports,TV,2013,7.359428,189
8,22199,Akame ga Kill!,Akame ga Kill!,Action|Adventure|Drama|Fantasy|Shounen,TV,2014,7.374775,26
9,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist:Brotherhood,Action|Adventure|Comedy|Drama|Fantasy|Magic|Mi...,TV,2009,9.148895,3


### 7.Recommendation based on Origin Year Distribution

In [99]:
anime['Origin_year_binned'] = pd.qcut(anime['Origin_year'], q = [0, .1, .25, .4, .6, .75, .9, 1], labels = [1,2,3,4,5,6,7])
anime['Origin_year_binned'] = anime['Origin_year_binned'].astype('int8')

In [100]:
def generating_anime_origin_year_distribution(user_agg, user, anime):
    """
    Generates the percentage distribution of anime origin years watched by each user (with at least 50 anime watched).

    Parameters:
    - user_agg (DataFrame): Aggregated user data
    - user (DataFrame): User-anime interaction data
    - anime (DataFrame): Anime metadata

    Returns:
    - origin_year_binned_df (DataFrame): Each user's percentage distribution of watched anime across origin year bins.
    - origin_year_binned_mean_dict (dict): Mean percentage distribution across all users for each origin year bin.
    """

    # Filter users who have watched at least 50 animes
    user_agg_50 = user_agg[user_agg['animes_watched'] >= 50]
    users_data_50 = user[user['user_id'].isin(user_agg_50['user_id'])]

    users_data_50 = users_data_50.copy()

    # Map origin year bins to user's anime history
    users_data_50['anime_origin_year_binned'] = users_data_50['anime_id'].map(
        anime.set_index('MAL_ID')['Origin_year_binned']
    )

    # Count animes watched per origin year bin per user
    origin_year_binned_df = users_data_50.groupby(['user_id', 'anime_origin_year_binned']).size()

    # Convert to percentage distribution per user
    origin_year_binned_df = (
        origin_year_binned_df / origin_year_binned_df.groupby(level=0).sum()
    ).reset_index().pivot(
        columns='anime_origin_year_binned', index='user_id', values=0
    ).fillna(0)

    # Rename and reset index
    origin_year_binned_df = origin_year_binned_df.rename_axis(None, axis=1)
    origin_year_binned_df = origin_year_binned_df.add_prefix('pct_origin_year_binned_')
    origin_year_binned_df.reset_index(inplace=True)

    # Calculate the mean distribution across all users
    origin_year_binned_mean_dict = origin_year_binned_df[origin_year_binned_df.columns[1:]].mean().to_dict()

    return origin_year_binned_df, origin_year_binned_mean_dict




In [101]:
origin_year_binned_df, origin_year_binned_mean_dict = generating_anime_origin_year_distribution(user_agg, user, anime)

In [102]:
origin_year_binned_df.head()

,user_id,pct_origin_year_binned_1,pct_origin_year_binned_2,pct_origin_year_binned_3,pct_origin_year_binned_4,pct_origin_year_binned_5,pct_origin_year_binned_6,pct_origin_year_binned_7
0,1,0.000000,0.076923,0.128205,0.179487,0.213675,0.179487,0.222222
1,3,0.009464,0.041009,0.186120,0.353312,0.309148,0.082019,0.018927
2,4,0.108333,0.283333,0.608333,0.000000,0.000000,0.000000,0.000000
3,6,0.075410,0.285246,0.436066,0.203279,0.000000,0.000000,0.000000
4,7,0.000000,0.044944,0.213483,0.584270,0.157303,0.000000,0.000000


In [103]:
origin_year_binned_mean_dict

{'pct_origin_year_binned_1': 0.043413609177278434,
 'pct_origin_year_binned_2': 0.11443863165776946,
 'pct_origin_year_binned_3': 0.21782759152870176,
 'pct_origin_year_binned_4': 0.25619821457173525,
 'pct_origin_year_binned_5': 0.21441812637273008,
 'pct_origin_year_binned_6': 0.09310651732700935,
 'pct_origin_year_binned_7': 0.060597309364775656}

####7.1 Anime recommendation based on origin year distribution

In [104]:
def anime_rec_on_origin_year_distribution(origin_year_binned_df, origin_year_binned_mean_dict, user_id, anime, user):
    """
    Recommends anime from a specific origin year bin that the user watches less compared to the overall average,
    promoting diversity in the user's watch history.

    Parameters:
    - origin_year_binned_df (DataFrame): User-level distribution of anime watched across origin year bins.
    - origin_year_binned_mean_dict (dict): Mean distribution across all users for each origin year bin.
    - user_id (int): The user ID for which recommendations are being made.
    - anime (DataFrame): Anime metadata including 'Origin_year_binned'.
    - user (DataFrame): User-anime interaction data.

    Returns:
    - DataFrame: Anime from the least-watched (by this user) year bin relative to the global average.
    """

    if user_id not in origin_year_binned_df['user_id'].values:
        print('Not enough Animes watched')
        return None

    # Get the user's distribution across origin year bins (excluding 'user_id' column)
    ind_year_binned_dist = origin_year_binned_df[origin_year_binned_df['user_id'] == user_id]\
        [origin_year_binned_df.columns[1:]].values[0]

    # Calculate the difference between the average and the user's distribution
    results = np.array(list(origin_year_binned_mean_dict.values())) - ind_year_binned_dist

    # Recommend anime from the bin with the greatest negative difference (i.e., least watched vs. average)
    target_bin = 1 + np.argmin(results)
    watched_animes = user[user['user_id'] == user_id]['anime_id'].tolist()
    recommendations = anime[(anime['Origin_year_binned'] == target_bin) & (~anime['MAL_ID'].isin(watched_animes))]

    return recommendations


In [105]:
rec_based_on_origin_year_dist = anime_rec_on_origin_year_distribution(origin_year_binned_df, origin_year_binned_mean_dict,
                                                                      selected_user_id_3, anime, user)
rec_based_on_origin_year_dist.head(10)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genres_edited,Origin_year,Popularity_adjusted,Origin_year_binned
2939,9260,Kizumonogatari I: Tekketsu-hen,8.4,Vampire,Kizumonogatari Part 1:Tekketsu,傷物語〈Ⅰ鉄血篇〉,Movie,1,"Jan 8, 2016",Unknown,...,7044.0,2424.0,1021.0,434.0,261.0,547.0,Vampire,2016,358,5
3098,10259,Da Yu Hai Tang (Movie),7.58,"Adventure, Supernatural, Drama, Romance",Big Fish & Begonia,大鱼海棠,Movie,1,"Jul 8, 2016",Unknown,...,1211.0,568.0,285.0,115.0,60.0,65.0,Adventure|Drama|Romance|Supernatural,2016,2896,5
3189,10687,Saint Seiya: Legend of Sanctuary,6.25,"Sci-Fi, Adventure, Fantasy, Shounen",Unknown,聖闘士星矢 LEGEND of SANCTUARY,Movie,1,"Jun 21, 2014",Unknown,...,2732.0,1726.0,1073.0,590.0,390.0,373.0,Adventure|Fantasy|Sci-Fi|Shounen,2014,3391,5
3245,10937,Mobile Suit Gundam: The Origin,8.36,"Action, Mecha, Military, Sci-Fi, Shounen, Space",Mobile Suit Gundam:The Origin,機動戦士ガンダムTHE ORIGIN,OVA,6,"Feb 28, 2015 to May 5, 2018",Unknown,...,716.0,259.0,140.0,56.0,35.0,234.0,Action|Mecha|Military|Sci-Fi|Shounen|Space,2015,2196,5
3269,11135,Hybrid Child,7.69,"Drama, Historical, Romance, Sci-Fi, Shounen Ai",Unknown,ハイブリッド チャイルド,OVA,4,"Oct 29, 2014 to Jan 28, 2015",Unknown,...,3420.0,1758.0,717.0,303.0,156.0,166.0,Ai|Drama|Historical|Romance|Sci-Fi|Shounen,2014,1965,5
3354,12017,Tiger & Bunny Movie 2: The Rising,7.81,"Action, Mystery, Comedy, Super Power",Tiger & Bunny:The Rising,劇場版 TIGER & BUNNY -The Rising-,Movie,1,"Feb 8, 2014",Unknown,...,731.0,273.0,117.0,28.0,14.0,32.0,Action|Comedy|Mystery|Super-Power,2014,3285,5
3430,12783,Sora no Otoshimono Final: Eternal My Master,6.77,"Comedy, Ecchi, Harem, Sci-Fi, Shounen, Superna...",Unknown,そらのおとしものＦｉｎａｌ永遠の私の鳥籠［エターナルマイマスター］,Movie,1,"Apr 26, 2014",Unknown,...,5768.0,3051.0,2440.0,1091.0,680.0,620.0,Comedy|Ecchi|Harem|Sci-Fi|Shounen|Supernatural,2014,1660,5
3452,13119,Hakuouki Movie 2: Shikon Soukyuu,7.77,"Action, Historical, Supernatural, Drama, Samur...",Hakuoki ~Demon of the Fleeting Blossom~ Warrio...,劇場版 薄桜鬼 第二章 士魂蒼穹,Movie,1,"Mar 8, 2014",Unknown,...,521.0,225.0,85.0,43.0,22.0,23.0,Action|Drama|Historical|Josei|Samurai|Supernat...,2014,3717,5
3490,13391,Rakuen Tsuihou,7.38,"Action, Sci-Fi, Mecha",Expelled from Paradise,楽園追放 -Expelled from Paradise-,Movie,1,"Nov 15, 2014",Unknown,...,4814.0,2130.0,856.0,399.0,196.0,116.0,Action|Mecha|Sci-Fi,2014,1619,5
3529,13843,Wild Adapter,6.24,"Action, Mystery, Drama, Shoujo",Wild Adapter,ワイルドアダプター,OVA,2,"Mar 26, 2014 to Sep 30, 2015",Unknown,...,477.0,316.0,129.0,59.0,32.0,57.0,Action|Drama|Mystery|Shoujo,2014,5164,5


####7.2 Combining this feature with model based prediction

In [106]:
# Can use the results with model based prediction
rec_based_on_origin_year_dist_and_model = model_rec_based_on_rating_prediction(selected_user_id_3, rec_based_on_origin_year_dist,
                                                                               user, encoded_dictionary, model)
rec_based_on_origin_year_dist_and_model.head(10)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,anime_id,anime_enc,rating_prediction
59,28851,88,9.291472
595,31043,56,9.287230
44,28805,1137,9.071254
53,32935,75,9.020352
108,33051,4305,8.938524
503,22319,183,8.917128
704,23283,1011,8.915527
538,22535,87,8.847038
359,25781,1664,8.834229
485,22199,52,8.713224


In [107]:
filter_recommendations(rec_based_on_origin_year_dist_and_model, anime, anime_agg).head(10)

,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,28851,Koe no Katachi,A Silent Voice,Drama|School|Shounen,Movie,2016,8.944652,28
1,31043,Boku dake ga Inai Machi,ERASED,Mystery|Psychological|Seinen|Supernatural,TV,2016,8.301554,29
2,28805,Bakemono no Ko,The Boy and the Beast,Adventure|Supernatural,Movie,2015,8.297841,457
3,32935,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,Haikyu!! 3rd Season,Comedy|Drama|School|Shounen|Sports,TV,2016,8.852343,124
4,33051,Mobile Suit Gundam: Iron-Blooded Orphans 2nd S...,Mobile Suit Gundam:Iron-Blooded Orphans 2nd Se...,Action|Drama|Mecha|Sci-Fi|Space,TV,2016,8.100126,1412
5,22319,Tokyo Ghoul,Tokyo Ghoul,Action|Drama|Horror|Mystery|Psychological|Sein...,TV,2014,7.659684,7
6,23283,Zankyou no Terror,Terror in Resonance,Mystery|Psychological|Thriller,TV,2014,8.056075,103
7,22535,Kiseijuu: Sei no Kakuritsu,Parasyte -the maxim-,Action|Drama|Horror|Psychological|Sci-Fi|Seinen,TV,2014,8.403520,38
8,25781,Shingeki no Kyojin: Kuinaki Sentaku,Unknown,Action|Fantasy|Shoujo,OVA,2014,8.275769,429
9,22199,Akame ga Kill!,Akame ga Kill!,Action|Adventure|Drama|Fantasy|Shounen,TV,2014,7.374775,26


### 8.Up for a Debate- Try divided opinion animes

*Finding Animes that have at least 25% bad ratings (ratings equal or below 5), \
at the same time have good ratings (ratings 8 or above) very much comparable to the bad ratings. \
(bad ratings/ good ratings) ratio is in between 0.9 to 1.15 range.*

In [108]:
anime_rating_freq = user.groupby(['anime_id','rating'])['rating'].count().unstack(fill_value = 0)
anime_rating_freq.head()

rating,1,2,3,4,5,6,7,8,9,10
anime_id,,,,,,,,,,
1,92,101,169,471,1258,3042,9059,19386,25340,27730
5,15,24,37,150,456,1419,5066,10885,10117,5448
6,62,62,128,385,1133,2939,8760,15499,13192,8119
7,33,44,100,281,806,1709,3348,3174,1489,640
8,7,17,32,81,175,305,510,393,162,107


In [109]:
print(anime_rating_freq.columns)
anime_rating_freq = anime_rating_freq.rename_axis(None, axis=1)
print(anime_rating_freq.columns)

anime_rating_freq = anime_rating_freq.add_prefix('rating_')

anime_rating_freq.reset_index( inplace = True)
anime_rating_freq.head()

Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='uint8', name='rating')
Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='uint8')


,anime_id,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10
0,1,92,101,169,471,1258,3042,9059,19386,25340,27730
1,5,15,24,37,150,456,1419,5066,10885,10117,5448
2,6,62,62,128,385,1133,2939,8760,15499,13192,8119
3,7,33,44,100,281,806,1709,3348,3174,1489,640
4,8,7,17,32,81,175,305,510,393,162,107


In [110]:
rating_columns = anime_rating_freq.columns[1:]
print(rating_columns)
bad_rating_columns  = anime_rating_freq.columns[1:6]
print(bad_rating_columns)
good_rating_columns = anime_rating_freq.columns[8:]
print(good_rating_columns)

Index(['rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
       'rating_7', 'rating_8', 'rating_9', 'rating_10'],
      dtype='object')
Index(['rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5'], dtype='object')
Index(['rating_8', 'rating_9', 'rating_10'], dtype='object')


In [111]:
anime_rating_freq['bad_rating_ratio'] = anime_rating_freq[bad_rating_columns].sum(axis=1) / anime_rating_freq[rating_columns].sum(axis=1)
anime_rating_freq['good_rating_ratio'] = anime_rating_freq[good_rating_columns].sum(axis=1) / anime_rating_freq[rating_columns].sum(axis=1)
anime_rating_freq.head()

,anime_id,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10,bad_rating_ratio,good_rating_ratio
0,1,92,101,169,471,1258,3042,9059,19386,25340,27730,0.024132,0.836211
1,5,15,24,37,150,456,1419,5066,10885,10117,5448,0.020287,0.786804
2,6,62,62,128,385,1133,2939,8760,15499,13192,8119,0.035204,0.732115
3,7,33,44,100,281,806,1709,3348,3174,1489,640,0.108741,0.456211
4,8,7,17,32,81,175,305,510,393,162,107,0.174399,0.370039


In [112]:
divided_opinion_cond = ((anime_rating_freq['bad_rating_ratio'] >= 0.25) & ((anime_rating_freq['bad_rating_ratio']/anime_rating_freq['good_rating_ratio']).between(0.9,1.15)))
divided_opinion_animes_mal_ids = anime_rating_freq[divided_opinion_cond]['anime_id'].values
print(len(divided_opinion_animes_mal_ids))

208


Then use model based prediction for recommendation

In [113]:
model_rec_based_on_rating_prediction(selected_user_id_3,
                                     anime[anime['MAL_ID'].isin(divided_opinion_animes_mal_ids)],
                                     user, encoded_dictionary, model).head(10)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


,anime_id,anime_enc,rating_prediction
192,33740,4449,7.640950
87,1262,2548,7.627176
178,13239,1299,7.499040
26,36439,4674,7.497042
159,41345,2960,7.463281
177,950,4263,7.456481
194,33743,2005,7.425953
50,33421,1051,7.384618
98,1793,4297,7.365943
147,21353,940,7.307616


In [114]:
filter_recommendations(model_rec_based_on_rating_prediction(selected_user_id_3,
                                     anime[anime['MAL_ID'].isin(divided_opinion_animes_mal_ids)],
                                     user, encoded_dictionary, model), anime, anime_agg).head(10)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


,MAL_ID,Name,English name,Genres_edited,Type,Origin_year,anime_avg_rating,Popularity_adjusted
0,33740,Katekyo Hitman Reborn! x ēlDLIVE Special,Unknown,Shounen|Super-Power,Special,2016,6.607708,4887
1,1262,Macross II: Lovers Again,Unknown,Adventure|Mecha|Military|Sci-Fi|Shounen|Space,OVA,1992,6.406743,4608
2,13239,Mass Effect: Paragon Lost,Mass Effect:Paragon Lost,Action|Adventure|Sci-Fi,Movie,2012,6.364092,5009
3,36439,Sword Art Online Movie: Ordinal Scale - Sword ...,Unknown,Comedy,Special,2017,6.461330,3688
4,41345,Noblesse,Unknown,Action|School|Supernatural|Vampire,TV,2020,6.440828,930
5,950,Street Fighter Alpha: Generations,Street Fighter Alpha:Generations,Adventure|Shounen,Movie,2005,6.507961,4332
6,33743,Fuuka,Fuuka,Drama|Ecchi|Music|Romance|School|Shounen,TV,2017,6.301728,652
7,33421,Hitori no Shita: The Outcast,Hitori no Shita - The Outcast,Action|Super-Power|Supernatural,TV,2016,6.386295,1179
8,1793,Kaze no Na wa Amnesia,A Wind Named Amnesia,Action|Dementia|Drama|Sci-Fi,Movie,1990,6.422173,4053
9,21353,Tokyo ESP,Tokyo ESP,Action|Comedy|Sci-Fi|Shounen|Super-Power|Super...,TV,2014,6.413659,1000
